In [1]:
#imports
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

C:\Users\Almog\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Almog\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Almog\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Almog\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
#getting the dataset
data=pd.read_csv(r"C:\Users\Almog\titanic_data.csv")

In [3]:
#showing the dataset
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#droping not concerned columns
def drop_not_concerned(data, columns):
    return data.drop(columns, axis=1)

columns = ["PassengerId","Name", "Ticket", "Fare", "Cabin", "Embarked"]
data = drop_not_concerned(data, columns)

In [5]:
# checking the empty spots in test dataset
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

In [6]:
# ignoting the empty spots in test dataset
data.dropna(inplace=True)

In [7]:
#shape of the data 
data.shape

(714, 6)

In [8]:
#spliting the Pclass into 3 diffrents columns
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix=column)], axis=1)
        data = data.drop(column, axis=1)
    return data


columns = ["Pclass"]
data=dummy_data(data, columns)

In [9]:
#changing the 'sex' column into 0,1 (0=female, 1=male)
def sex_to_int(data):
    le = LabelEncoder()
    le.fit(["male","female"])
    data["Sex"]=le.transform(data["Sex"]) 
    return data

data = sex_to_int(data)

In [10]:
#normalizing the data
def normalize_age(data):
    scaler = MinMaxScaler()
    data["Age"] = scaler.fit_transform(data["Age"].values.reshape(-1,1))
    return data
data = normalize_age(data)

In [11]:
# showing the data after the cleaning
data.head()

,Survived,Sex,Age,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3
0,0,1,0.271174,1,0,0,0,1
1,1,0,0.472229,1,0,1,0,0
2,1,0,0.321438,0,0,0,0,1
3,1,0,0.434531,1,0,1,0,0
4,0,1,0.434531,0,0,0,0,1


In [12]:
# setting up y label
y= data['Survived']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [101]:
# setting up x label, aka the features
X = data.drop(['Survived'],axis=1)
X.head()

,Sex,Age,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3
0,1,0.271174,1,0,0,0,1
1,0,0.472229,1,0,1,0,0
2,0,0.321438,0,0,0,0,1
3,0,0.434531,1,0,1,0,0
4,1,0.434531,0,0,0,0,1


In [102]:
# getting the x and y train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X ,y, test_size=0.25 ,random_state=1)

In [103]:
x_train.shape

(535, 7)

In [104]:
x_test.shape

(179, 7)

In [105]:
y_train.shape

(535,)

In [106]:
y_test.shape

(179,)

In [107]:
y_train = y_train.values.reshape((535,1))
y_test = y_test.values.reshape((179,1))

In [108]:
y_train.shape

(535, 1)

In [109]:
y_test.shape

(179, 1)

In [30]:
#simple logistic regression
features = 7
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([features,1]))
b = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(x,W) + b)
#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.add(tf.matmul(x, W), b), labels=y_))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=y_))

prediction = tf.round(y)
correct = tf.cast(tf.equal(prediction, y_), dtype=tf.float32)
accuracy = tf.reduce_mean(correct)

update = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(0,100000):
    sess.run(update, feed_dict = {x:x_train, y_:y_train})
    trainAcc = sess.run(accuracy, feed_dict={x: x_test, y_: y_test})
    if i % 10000 == 0 :
        print('Iteration:' , i , ' W:' , sess.run(W) , ' b:' , sess.run(b), ' loss:', loss.eval(session=sess, feed_dict = {x:x_train, y_:y_train}))
        print('accuracy',trainAcc)

Iteration: 0  W: [[-6.4730784e-04]
 [-1.9822811e-04]
 [-2.6903828e-04]
 [-1.3390415e-04]
 [ 2.6218560e-05]
 [-8.7790140e-05]
 [-4.5252059e-04]]  b: [-0.00051409]  loss: 0.7655632
accuracy 0.6256983
Iteration: 10000  W: [[-1.9747176 ]
 [-0.17084813]
 [-0.07076117]
 [ 0.18151355]
 [ 0.7325912 ]
 [ 0.12443284]
 [-1.1507663 ]]  b: [-0.29374278]  loss: 0.65959215
accuracy 0.7318436
Iteration: 20000  W: [[-2.9072788 ]
 [-0.12275955]
 [-0.04666103]
 [ 0.17684908]
 [ 1.2080294 ]
 [ 0.41913518]
 [-1.5592222 ]]  b: [0.06794298]  loss: 0.64446807
accuracy 0.8044693
Iteration: 30000  W: [[-3.5858989 ]
 [-0.12978537]
 [-0.06499355]
 [ 0.13005745]
 [ 1.4687114 ]
 [ 0.6472271 ]
 [-1.8182995 ]]  b: [0.29764152]  loss: 0.6373763
accuracy 0.8044693
Iteration: 40000  W: [[-4.109639  ]
 [-0.15258901]
 [-0.08540729]
 [ 0.09591416]
 [ 1.6457933 ]
 [ 0.8242004 ]
 [-2.0072246 ]]  b: [0.46275985]  loss: 0.6333355
accuracy 0.8044693
Iteration: 50000  W: [[-4.5327325 ]
 [-0.17982674]
 [-0.104333  ]
 [ 0.07161789

In [33]:
#adding hidden layer
features = 7
hidden_layer_nodes = 3
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W1 = tf.Variable(tf.truncated_normal([features,hidden_layer_nodes], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden_layer_nodes]))
z1 = tf.add(tf.matmul(x,W1),b1)
a1 = tf.nn.relu(z1)
W2 = tf.Variable(tf.truncated_normal([hidden_layer_nodes,1], stddev=0.1))
b2 = tf.Variable(0.)
z2 = tf.matmul(a1,W2) + b2
y = 1 / (1.0 + tf.exp(-z2))
loss = tf.reduce_mean(-(y_ * tf.log(y) + (1 - y_) * tf.log( 1 - y)))

prediction3 = tf.round(y)
correct3 = tf.cast(tf.equal(prediction3, y_), dtype=tf.float32)
accuracy3 = tf.reduce_mean(correct3)


update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(0,1000000):
    sess.run(update, feed_dict = {x:x_train, y_:y_train})
    trainAcc3 = sess.run(accuracy3, feed_dict={x: x_test, y_: y_test})
    if i % 100000 == 0 :
        print('Iteration:' , i , ' W:' , sess.run(W2) , ' b:' , sess.run(b2), ' loss:', loss.eval(session=sess, feed_dict = {x:x_train, y_:y_train}))
        print('accuracy',trainAcc3)


Iteration: 0  W: [[ 0.07643595]
 [ 0.15534988]
 [-0.01552953]]  b: -9.028719e-06  loss: 0.6967314
accuracy 0.37430167
Iteration: 100000  W: [[ 0.19761905]
 [ 0.19901118]
 [-0.12047128]]  b: -0.30990177  loss: 0.66822135
accuracy 0.6256983
Iteration: 200000  W: [[ 0.60525835]
 [ 0.50347954]
 [-0.5053719 ]]  b: -0.36393824  loss: 0.58094823
accuracy 0.7430168
Iteration: 300000  W: [[ 1.0097117 ]
 [ 0.8008462 ]
 [-0.95512825]]  b: -0.30128098  loss: 0.48409203
accuracy 0.7821229
Iteration: 400000  W: [[ 1.2175218 ]
 [ 0.95536596]
 [-1.0816977 ]]  b: -0.19435251  loss: 0.4609026
accuracy 0.7932961
Iteration: 500000  W: [[ 1.3380808]
 [ 1.051148 ]
 [-1.1288431]]  b: -0.1303511  loss: 0.4518862
accuracy 0.82122904
Iteration: 600000  W: [[ 1.4194533]
 [ 1.1168014]
 [-1.1611438]]  b: -0.090922125  loss: 0.44677916
accuracy 0.82122904
Iteration: 700000  W: [[ 1.4807997]
 [ 1.1653496]
 [-1.1923735]]  b: -0.062061608  loss: 0.44322568
accuracy 0.82122904
Iteration: 800000  W: [[ 1.5306638]
 [ 1.2

In [111]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

#input_dim is the Size of the vocabulary
#output_dim is the Dimension of the dense embedding.
model.add(layers.Embedding(input_dim=535, output_dim=5))

# Add a LSTM layer with 64 internal units.
model.add(layers.LSTM(64))

# Add a Dense layer with 5 units.
model.add(layers.Dense(5))

# Add an Activation layers 'relu'
model.add(layers.Activation('relu'))


model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)

model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=1, epochs=1
)

Train on 535 samples, validate on 179 samples
535/535 [==============================] - 8s 14ms/sample - loss: 0.7726 - acc: 0.5813 - val_loss: 0.6127 - val_acc: 0.6257
